In [1]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [2]:
train_all_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_all_df.reset_index(inplace=True)
train_all_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Store,Date,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-07-31,5,5263,555,1,1,0,1
1,2,2015-07-31,5,6064,625,1,1,0,1
2,3,2015-07-31,5,8314,821,1,1,0,1
3,4,2015-07-31,5,13995,1498,1,1,0,1
4,5,2015-07-31,5,4822,559,1,1,0,1


In [3]:
def featurize(df):
    df = df.copy()
    def featurizeStateHoliday(x):
        if x == 'a':
            return 1
        elif x == 'b':
            return 2
        elif x == 'c':
            return 3
        else:
            return x
    df['StateHoliday'] = df['StateHoliday'].map(lambda x: featurizeStateHoliday(x))
    

    def featurizeDate(ts): 
        features = {}
        features['cal:dayOfMonth'] = ts.day
        features['cal:dayOfWeek'] = ts.dayofweek
        features['cal:dayofyear'] = ts.dayofyear
        features['cal:month'] = ts.month
        features['cal:year'] = ts.year
        features['cal:weekofyear'] =  ts.weekofyear
        features['cal:quarter'] = ts.quarter

        return features

    date_features_series = df['Date'].map(lambda x: featurizeDate(x))
    date_features = DataFrame.from_records(date_features_series.tolist(), index=df['Date'])

    df = pandas.concat([df, date_features.reset_index()], axis=1)

    del df['Date']
    
    return df

In [4]:
train_all_df = featurize(train_all_df)

train_df, test_df = cross_validation.train_test_split(train_all_df, train_size=.90)

train_individual_df, train_ensemble_df = cross_validation.train_test_split(train_df, train_size=.75)

train_all_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
0,1,5,5263,555,1,1,0,1,31,4,212,7,3,31,2015
1,2,5,6064,625,1,1,0,1,31,4,212,7,3,31,2015
2,3,5,8314,821,1,1,0,1,31,4,212,7,3,31,2015
3,4,5,13995,1498,1,1,0,1,31,4,212,7,3,31,2015
4,5,5,4822,559,1,1,0,1,31,4,212,7,3,31,2015


In [5]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [8]:
train_df = train_df[['Sales', 'Store', 'DayOfWeek', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'cal:dayOfMonth', 'cal:dayOfWeek', 'cal:dayofyear', 'cal:month', 'cal:quarter', 'cal:weekofyear', 'cal:year']]

In [9]:
train_df.to_csv("train.csv", index=None)

In [10]:
train_df.head()

,Sales,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
977575,10262,506,2,1004,1,1,0,0,5,1,36,2,1,6,2013
418859,2551,405,6,351,1,0,0,0,21,5,172,6,2,25,2014
472494,0,520,7,0,0,0,0,0,4,6,124,5,2,18,2014
823722,0,523,7,0,0,0,0,0,23,6,174,6,2,25,2013
995116,0,207,7,0,0,0,0,0,20,6,20,1,1,3,2013
